In [21]:
import numpy as np
import pandas as pd 
from sklearn.cluster import KMeans
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
from bs4 import BeautifulSoup
import json
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim
import folium
import requests
from pandas.io.json import json_normalize
print("Libraries imported.")

Libraries imported.


In [3]:
crime_data=pd.read_csv('/home/lenovo/Downloads/coursera dataset/Crime_Incidents_in_2018.csv')
crime_data.head()



,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,BLOCK_GROUP,CENSUS_TRACT,VOTING_PRECINCT,LATITUDE,LONGITUDE,BID,START_DATE,END_DATE,OBJECTID,OCTO_RECORD_ID
0,-76.972215,38.852618,7006630,2018-07-26T00:00:00.000Z,MIDNIGHT,GUN,HOMICIDE,3100 - 3198 BLOCK OF 24TH STREET SE,402412.0,131645.0,...,007408 1,7408.0,Precinct 115,38.852610,-76.972212,NaN,2007-01-14T23:30:00.000Z,2007-01-14T00:00:00.000Z,427835575,07006630-01
1,-77.074960,38.912611,10954295,2018-04-05T21:17:59.000Z,EVENING,OTHERS,SEX ABUSE,3810 - 3899 BLOCK OF RESERVOIR ROAD NW,393499.0,138307.0,...,000201 1,201.0,Precinct 6,38.912603,-77.074958,NaN,2018-04-01T00:01:17.000Z,2018-04-05T21:18:34.000Z,427921626,10954295-01
2,-76.978422,38.899255,12146972,2018-07-31T00:00:00.000Z,MIDNIGHT,OTHERS,HOMICIDE,1700 - 1799 BLOCK OF BENNING ROAD NE,401872.0,136822.0,...,007901 1,7901.0,Precinct 81,38.899248,-76.978419,NaN,2012-10-18T00:10:00.000Z,2012-10-18T00:12:00.000Z,427980576,12146972-01
3,-76.965614,38.851265,17003498,2018-03-20T00:00:00.000Z,MIDNIGHT,GUN,HOMICIDE,3030 - 3299 BLOCK OF BUENA VISTA TERRACE SE,402985.0,131495.0,...,007502 2,7502.0,Precinct 115,38.851257,-76.965612,NaN,2017-01-06T19:01:52.000Z,2017-01-06T19:10:39.000Z,428132159,17003498-01
4,-76.929229,38.901839,16208156,2018-01-05T00:00:00.000Z,MIDNIGHT,GUN,HOMICIDE,4916 - 5067 BLOCK OF JAY STREET NE,406139.0,137111.0,...,007809 1,7809.0,Precinct 94,38.901832,-76.929227,NaN,2016-12-09T09:07:12.000Z,2016-12-09T09:11:51.000Z,428133121,16208156-01


In [4]:
col_list=crime_data.columns.tolist()
col_list

['X',
 'Y',
 'CCN',
 'REPORT_DAT',
 'SHIFT',
 'METHOD',
 'OFFENSE',
 'BLOCK',
 'XBLOCK',
 'YBLOCK',
 'WARD',
 'ANC',
 'DISTRICT',
 'PSA',
 'NEIGHBORHOOD_CLUSTER',
 'BLOCK_GROUP',
 'CENSUS_TRACT',
 'VOTING_PRECINCT',
 'LATITUDE',
 'LONGITUDE',
 'BID',
 'START_DATE',
 'END_DATE',
 'OBJECTID',
 'OCTO_RECORD_ID']

In [5]:
data=crime_data[['METHOD','OFFENSE','BLOCK','LATITUDE','LONGITUDE','WARD']]
data.head()


,METHOD,OFFENSE,BLOCK,LATITUDE,LONGITUDE,WARD
0,GUN,HOMICIDE,3100 - 3198 BLOCK OF 24TH STREET SE,38.852610,-76.972212,8
1,OTHERS,SEX ABUSE,3810 - 3899 BLOCK OF RESERVOIR ROAD NW,38.912603,-77.074958,2
2,OTHERS,HOMICIDE,1700 - 1799 BLOCK OF BENNING ROAD NE,38.899248,-76.978419,6
3,GUN,HOMICIDE,3030 - 3299 BLOCK OF BUENA VISTA TERRACE SE,38.851257,-76.965612,8
4,GUN,HOMICIDE,4916 - 5067 BLOCK OF JAY STREET NE,38.901832,-76.929227,7


In [6]:
all_types=set(data['OFFENSE'])
type_id=dict(zip(all_types,range(len(all_types))))
for i,h in type_id.items():
    data['OFFENSE'].replace(i,h,inplace=True)
all_types=set(data['BLOCK'])
type_id=dict(zip(all_types,range(len(all_types))))
for i,h in type_id.items():
    data['BLOCK'].replace(i,h,inplace=True)
all_types=set(data['METHOD'])
type_id=dict(zip(all_types,range(len(all_types))))
for i,h in type_id.items():
    data['METHOD'].replace(i,h,inplace=True)

/home/lenovo/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6786: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [7]:
from sklearn.cluster import KMeans

kclusters = 5

crimes = data.drop(['BLOCK'],1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(crimes)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 1, 3, 3, 3, 3, 3, 3, 0, 1], dtype=int32)

In [8]:
kmean=pd.DataFrame(kmeans.labels_)
data['CLUSTER']=""
col=['cluster']
kmean.columns=col
for i,h in kmean.iterrows():
    data['CLUSTER'].loc[i]=h[0]
data.drop(['WARD'],1,inplace=True)


/home/lenovo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/lenovo/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/home/lenovo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu

In [9]:
less_crimes=data['LATITUDE'].value_counts(ascending=True).head(20)
less_crime_latitudes=less_crimes.index.tolist()

new_data = data[data['LATITUDE'].isin(less_crime_latitudes) ]
new_data

,METHOD,OFFENSE,BLOCK,LATITUDE,LONGITUDE,CLUSTER
498,2,0,7164,38.962051,-77.056203,2
924,2,0,1497,38.956470,-77.065405,2
2007,1,1,5497,38.894568,-76.911551,0
2442,2,8,4249,38.904114,-77.002778,3
2814,2,0,264,38.922036,-77.031978,2
11011,2,8,6073,38.925808,-77.100827,1
11444,2,8,7249,38.955738,-77.025740,1
12877,2,8,1027,38.876070,-76.933954,3
15772,2,8,3452,38.937738,-77.071778,1
20947,2,5,6609,38.928441,-77.055899,1


In [10]:
safe_places=folium.Map(location=[38.9072, -77.0369],zoom_start=11)
for lat,lng,label in zip(new_data['LATITUDE'], new_data['LONGITUDE'], new_data['OFFENSE']):
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='red',
    fill=True,
    fill_color='#3186cc',
    fill_capacity=0.7,
    parse_html=False
    ).add_to(safe_places)
safe_places   
   

In [11]:

CLIENT_ID = '1RADDPZKPBM32VEDMKLVBUUJJUHZZFK5F3DEMVR5H4IUJZDO' # your Foursquare ID
CLIENT_SECRET = 'KPADLA0CICG0OZIRZI513I34VM51A2OX1FA2GOMOFKE2PWZR'  # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1RADDPZKPBM32VEDMKLVBUUJJUHZZFK5F3DEMVR5H4IUJZDO
CLIENT_SECRET:KPADLA0CICG0OZIRZI513I34VM51A2OX1FA2GOMOFKE2PWZR


In [12]:

radius = 500
LIMIT = 100

venues = []

for lat, long in zip(new_data['LATITUDE'], new_data['LONGITUDE']):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
 
            
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [13]:
venues_df = pd.DataFrame(venues)
venues_df

,0,1,2,3,4,5
0,38.962051,-77.056203,Rock Creek Park Nature Center and Planetarium,38.959913,-77.051744,Planetarium
1,38.962051,-77.056203,Fort DeRussy,38.960730,-77.056667,Other Great Outdoors
2,38.962051,-77.056203,Little Forest - Rock Creek Park,38.961563,-77.058630,Park
3,38.962051,-77.056203,WMATA Bus Stop #1003003,38.960938,-77.052463,Bus Stop
4,38.962051,-77.056203,Western Ridge Trail,38.960708,-77.051721,Trail
...,...,...,...,...,...,...
361,38.928371,-77.082921,Westchester Restaurant,38.928191,-77.079894,Restaurant
362,38.928371,-77.082921,Glover-Archibold Trail,38.929512,-77.080228,Trail
363,38.928371,-77.082921,4100 Cathedral Ave N2 Bus Stop,38.930031,-77.079132,Bus Stop
364,38.928371,-77.082921,Mass Avenue Deli,38.930196,-77.078692,Deli / Bodega


In [14]:
col=['a','b','address','latitude','longitude','category']
venues_df.columns=col
ve_df=venues_df
ve_df.head()

,a,b,address,latitude,longitude,category
0,38.962051,-77.056203,Rock Creek Park Nature Center and Planetarium,38.959913,-77.051744,Planetarium
1,38.962051,-77.056203,Fort DeRussy,38.960730,-77.056667,Other Great Outdoors
2,38.962051,-77.056203,Little Forest - Rock Creek Park,38.961563,-77.058630,Park
3,38.962051,-77.056203,WMATA Bus Stop #1003003,38.960938,-77.052463,Bus Stop
4,38.962051,-77.056203,Western Ridge Trail,38.960708,-77.051721,Trail


In [15]:

t_onehot = pd.get_dummies(ve_df[['category']], prefix="", prefix_sep="")
t_onehot['ADDRESS'] = ve_df['address'] 
t_onehot['latitude']=ve_df['a']
t_onehot['longitude']=ve_df['b']
columns = list(t_onehot.columns[-3:]) + list(t_onehot.columns[:-3])
t_onehot = t_onehot[columns]
t_onehot

,ADDRESS,latitude,longitude,ATM,African Restaurant,American Restaurant,Antique Shop,Arcade,Art Gallery,Asian Restaurant,...,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Rock Creek Park Nature Center and Planetarium,38.962051,-77.056203,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Fort DeRussy,38.962051,-77.056203,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Little Forest - Rock Creek Park,38.962051,-77.056203,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,WMATA Bus Stop #1003003,38.962051,-77.056203,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Western Ridge Trail,38.962051,-77.056203,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,Westchester Restaurant,38.928371,-77.082921,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
362,Glover-Archibold Trail,38.928371,-77.082921,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
363,4100 Cathedral Ave N2 Bus Stop,38.928371,-77.082921,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
364,Mass Avenue Deli,38.928371,-77.082921,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
washington_grouped= t_onehot.groupby(["ADDRESS"]).mean().reset_index()

print(washington_grouped.shape)
washington_grouped

(341, 158)


,ADDRESS,latitude,longitude,ATM,African Restaurant,American Restaurant,Antique Shop,Arcade,Art Gallery,Asian Restaurant,...,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,&pizza,38.908486,-77.058697,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2 Amys,38.937738,-77.071778,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,24 Hour Store,38.876070,-76.933954,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4100 Cathedral Ave N2 Bus Stop,38.928371,-77.082921,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,7-Eleven,38.878645,-76.996494,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336,YMCA Anthony Bowen,38.922036,-77.031978,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
337,Zion West African Restaurant,38.922533,-76.974791,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
338,barre3,38.904114,-77.002778,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
339,rag & bone,38.908486,-77.058697,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
safe_places_neighborhood=folium.Map(location=[38.9072, -77.0369],zoom_start=11)
for lat,lng,label in zip(ve_df['latitude'],ve_df['longitude'],ve_df['address']):
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='red',
    fill=True,
    fill_color='#3186cc',
    fill_capacity=0.7,
    parse_html=False
    ).add_to(safe_places_neighborhood)
safe_places_neighborhood   

In [33]:
clst=washington_grouped.drop('ADDRESS',axis=1)

In [34]:

kclusters = 5


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(clst)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 4, 0, 0, 0, 0, 0, 0, 1, 4], dtype=int32)

In [35]:
km=pd.DataFrame(kmeans.labels_)
clst['CLUSTER']=""
col=['cluster']
km.columns=col
for i,h in kmean.iterrows():
    clst['CLUSTER'].loc[i]=h[0]



/home/lenovo/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [36]:
clst

,latitude,longitude,ATM,African Restaurant,American Restaurant,Antique Shop,Arcade,Art Gallery,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Bowling Green,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Dessert Shop,Discount Store,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Food Truck,Forest,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gas Station,German Restaurant,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Historic Site,Hospital,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Jazz Club,Korean Restaurant,Latin American Restaurant,Laundromat,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Memorial Site,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Music Store,New American Restaurant,Nightclub,Nightlife Spot,Optical Shop,Other Great Outdoors,Other Repair Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Planetarium,Playground,Plaza,Pool,Recreation Center,Rental Car Location,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Skate Park,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spanish Restaurant,Sporting Goods Shop,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Thrift / Vintage Store,Tourist Information Center,Track,Trail,Train,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit,CLUSTER
0,38.908486,-77.058697,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
1,38.937738,-77.071778,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,38.876070,-76.933954,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
3,38.928371,-77.082921,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
4,38.878645,-76.996494,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
5,38.955738,-77.025740,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0

In [39]:
map_clusters = folium.Map(location=[38.9072, -77.0369], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, cluster in zip(clst['latitude'],clst['longitude'],clst['CLUSTER']):
    label = folium.Popup(' Cluster {}'.format(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters